
CS 410/510: NLP Final Project
Students: Patrick Niyongabo, Robert Handy, Hanin Alshalan

## Soccer Events Summarization and Results Prediction

### 0. Introdution and Imports

In [1]:
import nltk
import matplotlib.pyplot as plot 
import collections

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Data Processing

In [3]:
import pandas as pd
import io

path = '/content/drive/MyDrive/ColabData/kaggle-world-cup-2018-tweets.csv'
# filename = 'kaggle-world-cup-2018-tweets.csv'
df = pd.read_csv(path)
print(df.shape[0])

# df = pd.read_csv(io.StringIO(uploaded['kaggle-world-cup-2018-tweets.csv'].decode('utf-8')))
# df = pd.read_csv(io.StringIO('kaggle-world-cup-2018-tweets.csv').decode('utf-8')))
# df

530000


In [4]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>



1.1 Remove Low Quality Data

In [5]:
# Kinda just putting a couple filtering things here. It should probably get combined and cleaned up and inplace=True
# only_en = df.drop(df[df['lang'] != 'en'].index, inplace = False)
# longer_than_20 = df.drop(df[df['len'] < 20].index, inplace = False)

# lower case everything 
df["Tweet"] = df["Tweet"].str.lower()

# filter out short tweets (less than 50 chars)
df['Tweet'] = df['Tweet'].astype('str')
dd = df[df['Tweet'].apply(lambda x: len(x)>80)]
print(dd.shape[0])

195430


In [6]:
# create dictionary mapping games to tweets based on timestamps & hashtags
import json
json_file = open('/content/drive/MyDrive/ColabData/cupfinals.json') 
world_cup_games = json.load(json_file) 
#print(world_cup_games["URUPOR"]["finalscore"])
#print(type(world_cup_games))
world_cup_games["URUPOR"]["tweets"] = ["test tweet1", "test tweet2"]
#print(world_cup_games["URUPOR"])

# create dictionary mapping tweets by day
tweets_by_date = {}
for index, row in dd.iterrows():
  gametime = row["Date"]
  gamedate = gametime.split(" ")[0]
  if gamedate in tweets_by_date:
    tweets_by_date[gamedate].append(row['Tweet'])
  else:
    tweets_by_date[gamedate] = [row['Tweet']]
print(len(tweets_by_date["2018-07-15"]))  

59643


In [7]:
# print(len(tweets_by_date["2018-07-06"]))  # KeyError
# print(len(tweets_by_date["2018-07-05"])) # KeyError

for key in tweets_by_date:  # there are missing dates. i.e.: no tweets in our data even though games happened
  print(key + " : "+ str(len(tweets_by_date[key])))

2018-07-02 : 4251
2018-07-01 : 38687
2018-07-03 : 5970
2018-07-04 : 6925
2018-06-30 : 35541
2018-06-29 : 192
2018-07-10 : 28921
2018-07-11 : 15300
2018-07-15 : 59643


In [8]:
# assign tweets to a game if date of tweet match date of game
# more filtering can be added. example: match hashtag in tweet with match hashtag
for game in world_cup_games:
  gametime = world_cup_games[game]["starttime"]
  gamedate = gametime.split(" ")[0]
  print(game, gamedate)
  if gamedate in tweets_by_date:
    world_cup_games[game]["tweets"] = tweets_by_date[gamedate]
# print(len(world_cup_games["FRACRO"]["tweets"]))
# print(len(world_cup_games["CROENG"]["tweets"]))
assert len(tweets_by_date["2018-07-15"]) == len(world_cup_games["FRACRO"]["tweets"])
assert len(tweets_by_date["2018-07-11"]) == len(world_cup_games["CROENG"]["tweets"])

URUPOR 2018-06-30
FRAARG 2018-06-30
ESPRUS 2018-07-01
CRODEN 2018-07-01
BRAMEX 2018-07-02
BELJAP 2018-07-02
SWESUI 2018-07-03
COLENG 2018-07-03
URUFRA 2018-07-06
BRABEL 2018-07-06
RUSCRO 2018-07-07
SWEENG 2018-07-07
FRABEL 2018-07-10
CROENG 2018-07-11
BELENG 2018-07-14
FRACRO 2018-07-15


In [9]:
all_FRACRO_tweets = " ".join(world_cup_games["FRACRO"]["tweets"])
all_CROENG_tweets = " ".join(world_cup_games["CROENG"]["tweets"])
all_BRAMEX_tweets = " ".join(world_cup_games["BRAMEX"]["tweets"])
all_BELJAP_tweets = " ".join(world_cup_games["BELJAP"]["tweets"])
all_SWESUI_tweets = " ".join(world_cup_games["SWESUI"]["tweets"])
all_COLENG_tweets = " ".join(world_cup_games["COLENG"]["tweets"])
all_BEFORE_tweets = " ".join(tweets_by_date["2018-06-29"])
print("length of all_FRACRO_tweets = " + str(len(all_FRACRO_tweets)))
print("length of all_CROENG_tweets = " + str(len(all_CROENG_tweets)))
print("length of all_BRAMEX_tweets = " + str(len(all_BRAMEX_tweets)))
print("length of all_BELJAP_tweets = " + str(len(all_BELJAP_tweets)))
print("length of all_SWESUI_tweets = " + str(len(all_SWESUI_tweets)))
print("length of all_COLENG_tweets = " + str(len(all_COLENG_tweets)))
print("length of all_BEFORE_tweets = " + str(len(all_BEFORE_tweets)))

length of all_FRACRO_tweets = 6205172
length of all_CROENG_tweets = 1671357
length of all_BRAMEX_tweets = 439046
length of all_BELJAP_tweets = 439046
length of all_SWESUI_tweets = 631607
length of all_COLENG_tweets = 631607
length of all_BEFORE_tweets = 20374


In [10]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

stopwords = list(STOP_WORDS)
nlp = spacy.load('en')

# Place All As A Function For Reuseability
def text_summarizer(raw_docx):
    raw_text = raw_docx
    docx = nlp(raw_text)
    stopwords = list(STOP_WORDS)
    # Build Word Frequency
    # word.text is tokenization in spacy
    word_frequencies = {}  
    for word in docx:  
        if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1


    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    # Sentence Tokens
    sentence_list = [ sentence for sentence in docx.sents ]

    # Calculate Sentence Score and Ranking
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

    # Find N Largest
    summary_sentences = nlargest(3, sentence_scores, key=sentence_scores.get)
    final_sentences = [ w.text for w in summary_sentences ]
    summary = ' '.join(final_sentences)
    #print("Original Document\n")
    #print(raw_docx)
    #print("Total Length:",len(raw_docx))
    #print('\n\nSummarized Document\n')
    #print("Summary Length:",len(summary))
    #print("Summary: " + summary)
    return summary

In [11]:
# text_summarizer(all_FRACRO_tweets)
# text_summarizer(all_CROENG_tweets)
print("Summary of BRAMEX: " + text_summarizer(all_BRAMEX_tweets))
print("Summary of COLENG: " + text_summarizer(all_COLENG_tweets))

Summary of BRAMEX: all kasper schmeichel vs croatia saves penalty in extra time saves penalties in shootout still knocked out looking to celebrate your teams win at the fifa world cup russiatm kasper schmeichel vs croatia saves penalty in extra time saves penalties in shootout still knocked out of the route to final round of colombia quarter saves penalty in extra time saves penalties in shootout still knocked out of the route to final round of colombia quarter finals switzerland or sweden semi finals
Summary of COLENG: quarter final fixtures friday uruguay vs france brazil vs belgium saturday england vs retweet if your country are still in the russia uruguay france croatia sweden brazil belgiu southgate salute club and country leader intelligent diligent dignified respectful passionate rocks waistcoa quarter finals friday uruguay vs france brazil vs belgium saturday england vs swede england belgiu quarter finals friday uruguay vs france brazil vs belgium saturday england vs swede shef

### 2. Events Classification

In [12]:
# keyword # number of mentions # timestamp
# goal    # 100 tweets         # 20th june 2018 at 1pm

# 

### 3. Results Prediction

In [13]:
# manually using sentence templates
# automatically using text generator NLP tools

### 4. Events Summarization

In [14]:
# keyword # number of mentions # timestamp
# goal    # 100 tweets         # 20th june 2018 at 1pm

# 